# Stage 2: Nash equilibrium

In [9]:
using JuMP

In [35]:
function get_nash_primal(score_matrix)
    size = ndims(score_matrix)
    # for primal - P person
    m = Model()

    @variable(m, 1>= p[1:size] >=0)
    @variable(m, t)

    @constraint(m, sum(p) == 1)
    @constraint(m, min_const[k in 1:size], 
        sum(p[i] * score_matrix[i, k] for i=1:size) >= t)
    # Max min (g1, g2, ..., gn)
    @objective(m, Max, t)

#     println(m)
    solve(m)

    popt = getvalue(p)
#     println("Strategy: ", popt)
    println("E Score - Mine: ", getobjectivevalue(m))
    return popt
end
;

## 1. Morra problem

### Score function

### Demo data (2 fingers Morra)

In [26]:
size = 2
# calculate matrix incidences (max of mine is min of theirs or zero-sum game)
matrix_host = [2 -3; -3 4]
matrix_guest = [-2 3; 3 -4]
;

### Model

#### Host 

In [36]:
# Our strategy P
get_nash_primal(matrix_host)

E Score - Mine: -0.08333333333333359


2-element Array{Float64,1}:
 0.583333
 0.416667

#### Guest

In [37]:
get_nash_primal(matrix_guest)

E Score - Mine: 0.08333333333333359


2-element Array{Float64,1}:
 0.583333
 0.416667

## 2. Chess 

### Game score function

In [5]:
# the positional advantage and the ratings difference
# higher elo, first move better
function get_score_chess_game(player, opponent, is_first)
    if is_first
        x = player - opponent + 35
    else
        x = player - opponent - 35
    end
    return 1/(1 + 10^(-x/400)) 
end
;

### Match score function

In [14]:
function match_score(team_player, team_opponent, is_first)
    length = len(plan_player)
    if is_first
        return sum(get_score_chess_game(team_player[i], 
            team_opponent[i], i % 2 == 1) for i=1:length)
    else
        return sum(get_score_chess_game(team_player[i], 
            team_opponent[i], i % 2 == 0) for i=1:length)
    end
end
;

### Generate score matrices for all players

In [116]:
# one is enough!! No of games - 1's score = 2's score

### Nash equilibrium